In [8]:
!python -m spacy download en_core_web_sm
!pip install vaderSentiment
!pip install contractions
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 2.8 MB/s 
     |████████████████████████████████| 110 kB 54.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 82.0 MB/s 
     |████████████████████████████████| 7.6 MB 59.2 MB/s 


In [1]:
from collections import OrderedDict
import numpy as np
import spacy
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('punkt')
nlp = spacy.load('en_core_web_sm')
import string


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
# RAKE Implementation
import nltk
import string
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)

# https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents
def RakeExtract(text):
        text_tokens = nltk.word_tokenize(text)
        sub_tokens = []

        # delimiting tokenized sentence based on stop words
        for word in text_tokens:
            if word in STOP_WORDS or word in string.punctuation:
                sub_tokens.append("-")
            else:
                sub_tokens.append(word)


        spacetokens = []

        res = ""
        
        # more delimiation edge case handling
        # joining words together into phrases that are adjacent to each other in delimited sentence
        for i in range(0,len(sub_tokens)-1):
            if sub_tokens[i] == "-":
                if res != "":
                    spacetokens.append(res)
                spacetokens.append("-")
                res = ""
            if res == "" and sub_tokens[i] != "-":
                res = sub_tokens[i]

            if sub_tokens[i] != "-" and sub_tokens[i+1] != "-":
                res+= " " + sub_tokens[i+1]

        if sub_tokens[-1] != "-":
            res += sub_tokens[-1]
        spacetokens.append(res)

        # Keeping track of count of a specific word in wrdcnt dictionary
        wrdcnt = {}
        for i in sub_tokens:
            if i!= "-":
                wrdcnt[i] = wrdcnt.get(i,0)+1


        # This function initializes the vocabulary of the text
        # Returns a word dictionary initalized to a unique index for every word
        # This dictionary will store the index of the word in the 2-D Graph Adjacency List
        def getinitinfo(tokens):

            wordset = set()
            worddict = {}
            for i in tokens:
                if i != "-":
                    wordset.add(i)

            cnt = 0
            for i in wordset:
                worddict[i] = cnt
                cnt+=1
            return worddict

        # create graph to store counts of coocurrences of words within the same phrase in text
        dic = getinitinfo(sub_tokens)
        N = len(dic)
        graph = np.zeros((N,N))

        # Update graph with the adjacent words that appear together 
        for i in spacetokens:
            if i!= "-":
                spl = i.split()
                for i in range(len(spl)):
                    for j in range(i, len(spl)):
                        graph[dic[spl[i]]][dic[spl[j]]] +=1
                        if i!=j:
                            graph[dic[spl[j]]][dic[spl[i]]] +=1

        results = []

        # calculate Degree of word node in graph and the count of word in text
        # divide degree/wrdcnt to get imortance score for each word
        # caclulate important phrases score by adding score for each word in phrase
        degree = np.sum(graph, axis=0)
        for i, (k,v) in enumerate(dic.items()):

            results.append((k,degree[v]/wrdcnt[k]))

        results.sort(key=lambda x:x[1])
        results = results[::-1]

        keywords = set()
        for word in spacetokens:
            texspl = word.split()
            score = 0
            for x in results:
                for y in texspl:
                    if x[0] == y:
                        score += x[1] 
            keywords.add((word,score))

        # sort phrases by summation of their words score in increasing order
        keywords = list(keywords)
        keywords.sort(key=lambda x:x[1])
        keywords = keywords[::-1]
        keywords = keywords[:len(keywords)//3]

        return keywords


In [13]:
import contractions

text = '''
Poor customer experience and treatment received by banker Connie Walksler. I was expecting a warm, professional environment and it was definitely the contrary. This specific banker needs more training on wire transfers to Mexico especially, the lack of knowledge made me feel unsure about my transaction and business. She eventually helped me with this transfer but I left feeling not confident and uncomfortable.
'''

expanded_words = []   
for word in text.split():
  # using contractions.fix to expand the shortened words
  expanded_words.append(contractions.fix(word))  
   
expanded_text = ' '.join(expanded_words)
expanded_text = expanded_text.lower()
expanded_text = expanded_text.replace(',', '')

vals = RakeExtract(expanded_text)

In [14]:
# https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

for extract in vals:
          sid_obj = SentimentIntensityAnalyzer()
          sentiment_dict = sid_obj.polarity_scores(extract[0])
          
          if sentiment_dict['compound'] >= 0.10 :
            predicted_sentiments.append((extract , 'positive'))
 
          elif sentiment_dict['compound'] <= - 0.10 :
            predicted_sentiments.append((extract , 'negative'))
          else :
            predicted_sentiments.append((extract , 'neutral'))

predicted_sentiments

[(('poor customer experience', 9.0), 'negative'),
 (('banker connie walksler', 9.0), 'neutral'),
 (('specific banker needs', 9.0), 'neutral'),
 (('warm professional environment', 9.0), 'positive'),
 (('feel unsure', 4.0), 'negative'),
 (('treatment received', 4.0), 'neutral'),
 (('left feeling', 4.0), 'positive'),
 (('poor customer experience', 9.0), 'negative'),
 (('banker connie walksler', 9.0), 'neutral'),
 (('specific banker needs', 9.0), 'neutral'),
 (('warm professional environment', 9.0), 'positive'),
 (('feel unsure', 4.0), 'negative'),
 (('treatment received', 4.0), 'neutral'),
 (('left feeling', 4.0), 'positive')]